In [ ]:
!git clone https://github.com/twmmason/disco

Cloning into 'disco'...
remote: Enumerating objects: 3677, done.
remote: Counting objects: 100% (575/575), done.
remote: Compressing objects: 100% (365/365), done.
remote: Total 3677 (delta 269), reused 456 (delta 187), pack-reused 3102
Receiving objects: 100% (3677/3677), 179.06 MiB | 35.34 MiB/s, done.
Resolving deltas: 100% (961/961), done.


In [ ]:
%cd /content/disco
!git checkout generator_test

/content/disco
Checking out files: 100% (2736/2736), done.
Branch 'generator_test' set up to track remote branch 'generator_test' from 'origin'.
Switched to a new branch 'generator_test'


In [ ]:
!pip install piq ftfy ipywidgets lpips pytorch_lit pandas timm pip pytorch-lightning einops omegaconf flask_cors dependency-injector twilio encoders dill dalle2_pytorch matplotlib opencv-python

In [ ]:
!python install.py

Cloning into 'CLIP'...
remote: Enumerating objects: 222, done.
remote: Total 222 (delta 0), reused 0 (delta 0), pack-reused 222
Receiving objects: 100% (222/222), 8.91 MiB | 29.63 MiB/s, done.
Resolving deltas: 100% (113/113), done.

Obtaining file:///content/disco/lib/CLIP
  Running setup.py develop for clip
Cloning into 'ResizeRight'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 109 (delta 9), reused 8 (delta 8), pack-reused 98
Receiving objects: 100% (109/109), 37.47 KiB | 4.16 MiB/s, done.
Resolving deltas: 100% (64/64), done.

Cloning into 'pytorch3d-lite'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 55 (delta 30), reused 38 (delta 15), pack-reused 0
Unpacking objects: 100% (55/55), done.

Cloning into 'MiDaS'...
remote: Enumerating objects: 498, done.
remote: Counting objec

In [ ]:
# If in a low GPU memory environment, make the following changes
# Change modules/generators/generator_ld/generator.py line 588 self.args.clip_guidance=False, line 414 and 421 map_location="cpu"

In [ ]:
import os, sys
PROJECT_DIR=os.getcwd()
sys.path.append(f'{PROJECT_DIR}/lib/gl_3_xl')
sys.path.append(f'{PROJECT_DIR}/lib/CLIP')
sys.path.append(f'{PROJECT_DIR}/lib/MiDaS')
sys.path.append(f'{PROJECT_DIR}/lib/AdaBins')
sys.path.append(f'{PROJECT_DIR}/lib/latent-diffusion')
sys.path.append(f'{PROJECT_DIR}/lib/ResizeRight')
sys.path.append(f'{PROJECT_DIR}/lib/pytorch3d-lite')

import argparse
import json
from types import SimpleNamespace
from modules.manager.chain.chain import Chain
from modules.manager.projects.api import Api
from modules.manager.projects.project import Project


/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
from importlib import reload
import modules.manager.chain.chain
reload(modules.manager.chain.chain)
from modules.manager.chain.chain import Chain

In [ ]:
def run_custom():
    # Empty prompt will generate a 4-color grid, URL of image in prompt will pull down that image
    prompt = ""
    # prompt = "https://cdn.discordapp.com/attachments/970060442774958151/970073453052985445/BeepleBrandingConsultancy-PANCE-9.png"

    arbitrary_code_to_run = """
# Converts output of previous generator to black and white (dithering)
# and adds prompt as text in top left corner
import numpy as np
import os
from PIL import Image
from PIL import ImageDraw

im = Image.open(init_image)
im = im.convert('1')
draw = ImageDraw.Draw(im)
draw.text((0, 0), prompt, (255))

filename_out = f"{i_generator}_arbitrary.png"
im.save(os.path.join("content/output", filename_out))
im.save(os.path.join("static/output", filename_out))
    """
    project = Project(1)
    project.generators = [
        SimpleNamespace(
            **{
                "type": 0,
                "enabled": True,
                "settings": {
                    "prompt":prompt,
                    "reflect": True,
                },
            }
        ),
        SimpleNamespace(
            **{
                "type": 0,
                "enabled": True,
                "settings": {
                    "prompt":"this prompt will be ignored because it's a chain",
                    "reflect": True,
                },
            }
        ),
        SimpleNamespace(
            **{
                "type": 5,
                "enabled": True,
                "settings": {
                    "prompt": "Hello World",
                    "code": arbitrary_code_to_run,
                },
            }
        ),
    ]

    print("running project chain...")
    chain = Chain()
    chain.filename = chain.run_project(project)
run_custom()

running project chain...
Using device: cpu
Running project 1: 
namespace(enabled=True, settings={'prompt': '', 'reflect': True, 'init_image': '', 'i_generator': 0}, type=0) 0_test.png static/data/projects/1/output/0/ 0_test.png
Saving Updated [namespace(enabled=True, output_path='data/projects/1/output/0//0_test.png', settings={'prompt': '', 'reflect': True, 'init_image': '', 'i_generator': 0}, type=0), namespace(enabled=True, settings={'prompt': "this prompt will be ignored because it's a chain", 'reflect': True}, type=0), namespace(enabled=True, settings={'prompt': 'Hello World', 'code': '\n# print(init_image)\n# Converts output of previous generator to black and white (dithering)\n# and adds prompt as text in top left corner\nimport numpy as np\nimport os\nfrom PIL import Image\nfrom PIL import ImageDraw\n\nim = Image.open(init_image)\nim = im.convert(\'1\')\ndraw = ImageDraw.Draw(im)\ndraw.text((0, 0), prompt, (255))\n\nfilename_out = f"{i_generator}_arbitrary.png"\nim.save(os.path